# 01: Code Normalization

This notebook demonstrates code normalization - the critical first step that makes plagiarism detection robust to cosmetic changes.

In [ ]:
import sys
sys.path.insert(0, '..')

import matplotlib.pyplot as plt
import seaborn as sns
from src.normalization import get_normalizer, PythonNormalizer
from src.io import load_submissions
from IPython.display import display, HTML

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Before vs After: Python Normalization

In [ ]:
# Sample code with various cosmetic differences
original_code = '''
def calculate_fibonacci(number):
    """Calculate fibonacci number recursively."""
    # Base case
    if number <= 1:
        return number
    # Recursive case
    return calculate_fibonacci(number - 1) + calculate_fibonacci(number - 2)
'''

# Initialize normalizer
normalizer = PythonNormalizer()

# Normalize
normalized_code = normalizer.normalize(original_code)

# Display side-by-side
print("=" * 80)
print("BEFORE NORMALIZATION")
print("=" * 80)
print(original_code)
print("\n" + "=" * 80)
print("AFTER NORMALIZATION")
print("=" * 80)
print(normalized_code)

## 2. Identifier Mapping

See how variable and function names are normalized to canonical forms.

In [ ]:
# Get identifier mapping
identifier_map = normalizer.get_identifier_map()

print("📋 Identifier Mapping")
print("=" * 50)
print(f"{'Original Name':<30} {'Normalized Name':<20}")
print("=" * 50)

for original, normalized in identifier_map.items():
    print(f"{original:<30} {normalized:<20}")

print(f"\nTotal identifiers normalized: {len(identifier_map)}")

## 3. Testing with Disguised Plagiarism

Let's test with two codes that are identical in logic but look different.

In [ ]:
# Original code
code1 = '''
def factorial(n):
    if n <= 1:
        return 1
    return n * factorial(n-1)
'''

# "Disguised" version - renamed variables, added comments
code2 = '''
def compute_fact(num):
    # Base case for recursion
    if num <= 1:
        return 1
    # Recursive multiplication
    return num * compute_fact(num - 1)
'''

# Normalize both
norm1 = normalizer.normalize(code1)
normalizer.reset_counters()
norm2 = normalizer.normalize(code2)

print("🔍 Disguised Plagiarism Test")
print("=" * 80)
print("\nORIGINAL CODE 1:")
print(code1)
print("\nDISGUISED CODE 2:")
print(code2)
print("\n" + "=" * 80)
print("NORMALIZED CODE 1:")
print(norm1)
print("\nNORMALIZED CODE 2:")
print(norm2)
print("\n" + "=" * 80)

# Check if identical after normalization
if norm1.strip() == norm2.strip():
    print("\n✅ SUCCESS: Normalized codes are IDENTICAL!")
    print("   Normalization successfully removed cosmetic differences.")
else:
    print("\n⚠️  Normalized codes differ slightly (but are very similar)")

## 4. Normalization Effect on Code Length

In [ ]:
# Load sample submissions
submissions = load_submissions('../data/raw/sample_submissions.csv')

# Normalize all and compare lengths
original_lengths = []
normalized_lengths = []
submission_ids = []

for sub in submissions:
    original_lengths.append(len(sub['code']))
    
    normalizer.reset_counters()
    normalized = normalizer.normalize(sub['code'])
    normalized_lengths.append(len(normalized))
    
    submission_ids.append(sub['submission_id'])

# Visualize
x = range(len(submission_ids))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar([i - width/2 for i in x], original_lengths, width, 
               label='Original', color='steelblue', alpha=0.8)
bars2 = ax.bar([i + width/2 for i in x], normalized_lengths, width,
               label='Normalized', color='coral', alpha=0.8)

ax.set_xlabel('Submission ID', fontsize=12, fontweight='bold')
ax.set_ylabel('Code Length (characters)', fontsize=12, fontweight='bold')
ax.set_title('Code Length: Before vs After Normalization', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(submission_ids)
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics
avg_reduction = (sum(original_lengths) - sum(normalized_lengths)) / len(submissions)
print(f"\n📊 Average length reduction: {avg_reduction:.1f} characters")
print(f"   (Comments, docstrings, and whitespace removed)")

## 5. Testing Different Normalizations

In [ ]:
test_code = '''
class BinarySearch:
    """Binary search implementation."""
    
    def search(self, array, target):
        # Initialize pointers
        left_pointer = 0
        right_pointer = len(array) - 1
        
        while left_pointer <= right_pointer:
            middle = (left_pointer + right_pointer) // 2
            
            if array[middle] == target:
                return middle
            elif array[middle] < target:
                left_pointer = middle + 1
            else:
                right_pointer = middle - 1
        
        return -1  # Not found
'''

# Normalize
normalizer.reset_counters()
normalized = normalizer.normalize(test_code)

print("📝 Complex Code Normalization Example")
print("=" * 80)
print("\nORIGINAL:")
print(test_code)
print("\n" + "=" * 80)
print("NORMALIZED:")
print(normalized)
print("\n" + "=" * 80)

# Show identifier mapping
print("\nIdentifier Mapping:")
for orig, norm in normalizer.get_identifier_map().items():
    print(f"  {orig:20} → {norm}")

## 6. Visualization: Normalization Components

In [ ]:
# Count what gets removed
sample_code = original_code

# Count components
comment_lines = sample_code.count('#')
docstring_count = sample_code.count('"""') // 2
blank_lines = sample_code.count('\n\n')
total_lines = sample_code.count('\n') + 1
code_lines = total_lines - comment_lines - docstring_count - blank_lines

# Pie chart
components = ['Code Lines', 'Comments', 'Docstrings', 'Blank Lines']
counts = [code_lines, comment_lines, docstring_count, blank_lines]
colors = ['#2ecc71', '#e74c3c', '#3498db', '#95a5a6']
explode = (0.05, 0, 0, 0)

plt.figure(figsize=(10, 8))
plt.pie(counts, labels=components, autopct='%1.1f%%', startangle=90,
        colors=colors, explode=explode, shadow=True, textprops={'fontsize': 12, 'fontweight': 'bold'})
plt.title('Code Composition Before Normalization', 
          fontsize=16, fontweight='bold', pad=20)
plt.axis('equal')
plt.show()

print("\n🎯 Normalization removes everything except actual code logic!")

## Summary

✅ **Comment Removal**: All comments and docstrings removed

✅ **Identifier Normalization**: Variables → var_1, var_2; Functions → func_1, func_2

✅ **Whitespace Normalization**: Consistent formatting

✅ **Cosmetic-Proof**: Disguised plagiarism becomes obvious after normalization

**Key Insight**: Normalization is what makes the system robust to trivial changes like renaming variables or adding comments. Two codes with identical logic will look nearly identical after normalization!

**Next Steps**: Proceed to notebook 02 to see lexical similarity analysis!